# Notes

This notebook was designed to run on Google colab and reads stored copies of our source data from shared Google drive folders. We were unsuccessful at running similar notebooks on our full datasets on an AWS server implementations of the code, but Google colab managed to render the plotly charts without a problem.

Other than the setup and data imports, all other versions of the graph we used in the presentation (4d) were kept in the notebook for reference, but they were commented out to increase speed in repeated running.

# Setup

## Installs and Imports

In [1]:
# pip install --upgrade pip --quiet

In [2]:
# pip install -U kaleido # only needed for png image exports of graphs for Graph 5

In [3]:
import os
import gc
import itertools
from datetime import datetime, timedelta
from google.colab import drive

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.subplots import make_subplots

In [4]:
# Needed for s3 integration; not needed for colab introduction
# import boto3
# from sagemaker import get_execution_role

# role = get_execution_role()

## Load Data

In [5]:
USER = 'Tom'
TESTING = False

In [6]:
# Mount drive where you will do your work.
drive.mount('/content/drive')
if USER == 'Tom': 
  root_dir = "/content/drive/My Drive/W210/"
  project_folder = "Capstone/"
elif USER == 'Bri':
  pass
  root_dir = "/content/drive/My Drive/W210/"
  project_folder = "Capstone/Data/"
elif USER == 'Bronte':
  pass
  # root_dir = "/content/drive/My Drive/266/"
  # project_folder = "Final Project/"
elif USER == 'Kurt':
  pass
  root_dir = "/content/drive/My Drive/210/"
  project_folder = "Data/"
else:
  raise Exception("User unrecognized, must connect to shared drive")

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  # to test if all is working, you can uncomment these two lines below--it should write a file to the shared drive
  # !touch 'new_file_in_working_directory.txt'
# print('\nYour working directory was changed to ' + root_dir + project_folder + \
#         "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

os.chdir(root_dir + project_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Read data with predicted carbon intensity
# bucket='ucb-mids-capstone/Data/EIA'
# forecasts_data_key = 'co2_intensity_combined_fcsts_final.csv.csv'
# data_location = f's3://{bucket}/{forecasts_data_key}'
forecasts_df=pd.read_csv('Data/Aggregated_Data/co2_intensity_combined_fcsts_final.csv',  index_col='utc_date', parse_dates=True)
forecasts_df

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
utc_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,0.470492,0.491854,0.522422,0.527823,0.530451,0.537607,0.542250,0.546962,0.550348,0.550521,...,0.644654,0.641758,0.582585,0.519683,0.483774,0.470625,0.466023,0.468389,0.477112,0.502266
2022-01-02,0.477705,0.537870,0.563954,0.572643,0.577694,0.584193,0.586832,0.589021,0.591147,0.589424,...,0.648259,0.646650,0.586942,0.517407,0.477475,0.467447,0.466135,0.467828,0.475757,0.507616
2022-01-03,0.477013,0.542351,0.576605,0.572354,0.574267,0.580528,0.586977,0.589649,0.591309,0.590583,...,0.645253,0.637235,0.575053,0.510993,0.477432,0.461118,0.455679,0.460788,0.467711,0.508481
2022-01-04,0.500176,0.543726,0.573262,0.569135,0.568175,0.566668,0.565902,0.561390,0.556640,0.558539,...,0.607180,0.593095,0.533867,0.454161,0.420935,0.402075,0.394283,0.392358,0.393509,0.407838
2022-01-05,0.463086,0.522199,0.565619,0.566704,0.568747,0.573697,0.573939,0.574462,0.574837,0.574203,...,0.525286,0.509610,0.428085,0.345803,0.327854,0.320065,0.324047,0.328244,0.347742,0.385012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.628688,0.653858,0.653038,0.650209,0.654006,0.659589,0.662665,0.667334,0.663145,0.657335,...,0.657670,0.641683,0.613931,0.564681,0.527258,0.512631,0.506401,0.501162,0.502057,0.535347
2022-12-28,0.565477,0.565661,0.566580,0.559041,0.560466,0.560109,0.559592,0.556818,0.557277,0.556927,...,0.655575,0.645370,0.625490,0.545905,0.501468,0.477549,0.467542,0.462824,0.464558,0.483404
2022-12-29,0.431034,0.517039,0.540298,0.559652,0.572439,0.585651,0.600079,0.608849,0.614148,0.622391,...,0.616109,0.598318,0.570703,0.492857,0.440844,0.417223,0.399269,0.396837,0.402298,0.428609


In [8]:
# df_gen=pd.read_csv('s3://ucb-mids-capstone/Data/Aggregated_Data/train_val_test_w_src_fcsts.csv', index_col='utc_time', parse_dates=True)
df = pd.read_csv('Data/Aggregated_Data/train_val_test_w_src_fcsts.csv', index_col='utc_time', parse_dates=True)

In [9]:
df['offset'] = df.apply(lambda row: 8 if (row['utc_hour'] + row['local_hour']) % 2 == 0 else 7, axis=1)
offset_column = df.pop('offset')
df.insert(2, 'offset', offset_column)

In [10]:
df

,utc_time_with_offset,local_time,offset,local_year,local_month,local_week_of_year,local_day,local_day_of_week,local_day_of_year,local_is_weekend,...,total_weighted_radiation_now,total_weighted_radiation_1_day_ahead,total_weighted_radiation_2_days_ahead,total_weighted_radiation_3_days_ahead,total_weighted_radiation_4_days_ahead,ng_ng_fcst,ng_sun_fcst,ng_wnd_fcst,ng_wat_fcst,ng_col_fcst
utc_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2018-12-31 16:00:00,8,2018,12,1,31,0,365,False,...,268.174038,72.126662,117.757422,120.941605,116.544292,7805.148451,1600.952689,2797.653043,2613.926821,1071.357559
2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2018-12-31 17:00:00,8,2018,12,1,31,0,365,False,...,0.283649,0.181061,0.256713,0.325910,0.383436,9267.077871,1392.607499,2927.866663,3630.647469,1351.668993
2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2018-12-31 18:00:00,8,2018,12,1,31,0,365,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,9678.001598,0.000000,2819.551921,4274.134951,1592.408450
2019-01-01 03:00:00,2019-01-01 03:00:00+00:00,2018-12-31 19:00:00,8,2018,12,1,31,0,365,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,10025.806901,0.000000,2740.122328,4428.710828,1606.862852
2019-01-01 04:00:00,2019-01-01 04:00:00+00:00,2018-12-31 20:00:00,8,2018,12,1,31,0,365,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,10045.981002,0.000000,2642.982183,4095.323760,1567.627902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-04 19:00:00,2023-01-04 19:00:00+00:00,2023-01-04 11:00:00,8,2023,1,1,4,2,4,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,12351.726902,0.000000,2319.810522,1930.760452,1109.512899
2023-01-04 20:00:00,2023-01-04 20:00:00+00:00,2023-01-04 12:00:00,8,2023,1,1,4,2,4,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,12336.828771,0.000000,2562.236142,1877.956797,989.298593
2023-01-04 21:00:00,2023-01-04 21:00:00+00:00,2023-01-04 13:00:00,8,2023,1,1,4,2,4,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,12568.360252,0.000000,2834.838160,1858.736902,927.374862


In [11]:
# Read just data with actual carbon intensity
df_gen = df.loc['2021-12-25 08:00:00':'2023-01-05 07:00:00', ['co2_emissions_intensity_for_consumed_electricity', 'local_time', 'offset']]
df_gen.head(5)

,co2_emissions_intensity_for_consumed_electricity,local_time,offset
utc_time,,,
2021-12-25 08:00:00,0.519489,2021-12-25 00:00:00,8
2021-12-25 09:00:00,0.518672,2021-12-25 01:00:00,8
2021-12-25 10:00:00,0.514867,2021-12-25 02:00:00,8
2021-12-25 11:00:00,0.517689,2021-12-25 03:00:00,8
2021-12-25 12:00:00,0.508942,2021-12-25 04:00:00,8


In [12]:
leading_window=2*24

## Prepare dataframes for forecasts

In [13]:
# This function takes a forecast date, and selects the data for a window around it.
# Each window will be used for plotting one day of the data slider
def update_plot(forecast_date):
    # select carbon intensity from that window
    left_window=forecast_date - timedelta(hours=leading_window)
    right_window=forecast_date + timedelta(hours=96) 
    forecast_start = forecast_date + timedelta(hours=1)
    df_day = df_gen.loc[left_window:right_window, ['co2_emissions_intensity_for_consumed_electricity', 'local_time', 'offset']]
    
    # join the forecast carbon intensity for that window
    forecasts = forecasts_df.loc[forecast_date].squeeze()
    datetime_index = pd.date_range(forecast_start, periods=96, freq='1H')
    forecasts.index = datetime_index
    df_day['co2_forecast'] = forecasts
    df_day = df_day.reindex(columns=['local_time', 'offset', 'co2_emissions_intensity_for_consumed_electricity', 'co2_forecast'])
    return df_day

In [14]:
# Until memory issues resolved, select only 100 steps of the year for now
subsample=len(forecasts_df)
# subsample=100

# Create list of dfs (1 for each day of slider plot)
list_of_dfs = []

# Used forecast index here because it goes from 1/1/22-12/31/22 without pesky 00:00:00 suffixes
# since we are using 24 hour strikes anyway.  You will have 365 dfs, one for each day.
for f in forecasts_df.iloc[0 : subsample].index:
    carbon_day_df = update_plot(f)
    carbon_day_df = carbon_day_df.astype({c: np.float32 for c in carbon_day_df.select_dtypes(include='float64').columns})
    list_of_dfs.append(carbon_day_df)
print(f"Number of separate dfs (separate timesteps on the plot): {len(list_of_dfs)}")

Number of separate dfs (separate timesteps on the plot): 365


In [15]:
list_of_dfs[0]

,local_time,offset,co2_emissions_intensity_for_consumed_electricity,co2_forecast
utc_time,,,,
2021-12-30 00:00:00,2021-12-29 16:00:00,8,0.638096,NaN
2021-12-30 01:00:00,2021-12-29 17:00:00,8,0.607151,NaN
2021-12-30 02:00:00,2021-12-29 18:00:00,8,0.619299,NaN
2021-12-30 03:00:00,2021-12-29 19:00:00,8,0.622238,NaN
2021-12-30 04:00:00,2021-12-29 20:00:00,8,0.628751,NaN
...,...,...,...,...
2022-01-04 20:00:00,2022-01-04 12:00:00,8,0.347700,0.470625
2022-01-04 21:00:00,2022-01-04 13:00:00,8,0.336308,0.466023
2022-01-04 22:00:00,2022-01-04 14:00:00,8,0.346974,0.468389


In [16]:
def get_timestamp_edges(dt_timestamps):
    # initialize a list to store the selected timestamps
    selected_timestamps = []

    # add the first timestamp to the selected list
    selected_timestamps.append(dt_timestamps[0])

    # iterate through the remaining timestamps
    for i in range(1, len(dt_timestamps)-1):
        # check if the previous timestamp is not one hour prior
        if (dt_timestamps[i] - dt_timestamps[i-1] != timedelta(hours=1)):
            # if True, that means dt_timestamps[i-1] was an endpoint (and add an hour)
            selected_timestamps.append(dt_timestamps[i-1] + timedelta(hours=1))
            # and the current timestamp is the start of a new consecutive range
            selected_timestamps.append(dt_timestamps[i])

    # add the last timestamp to the selected list
    selected_timestamps.append(dt_timestamps[-1])
    return(selected_timestamps)

In [17]:
def get_rectangle_edges(selected_timestamps):
    x_vals = []
    y_vals = []
    # num_shapes = len(selected_timestamps)/2

    for i, t in enumerate(selected_timestamps):
      if i % 2 == 0:
        y_vals.append(0)
        y_vals.append(2)
        y_vals.append(2)
        y_vals.append(0)
        y_vals.append(0)
        y_vals.append(None)

      if i % 2 == 0:
        x_vals.append(t)
        x_vals.append(t)
        x_vals.append(t)
      if i % 2 == 1:
        x_vals.insert(-1, t)
        x_vals.insert(-1, t)
        x_vals.append(None)

    # convert x_vals to timestamps for plotting
    # x_vals = [ts.to_pydatetime() for ts in x_vals if ts is not None]

    # remove trailing "None"
    x_vals.pop(-1)
    y_vals.pop(-1)
    
    return x_vals, y_vals

In [18]:
from pandas._libs.tslibs.fields import get_timedelta_field
##############################################################################################
# This next section simply calculates a list of carbon intensity reductions in percentage from 
# shifting from the highest quartiles to the lowest quartiles of carbon intensity
quartile_intensities = []

for d in list_of_dfs:
    # Create quartile dataframe that identifies which datetimes we want populated in chart
    val_df = d.iloc[-96:193].copy()

    # Reset index so able to plot columns
    val_df.reset_index(inplace=True)

    # Identify quartiles for carbon intensity
    val_df['quartile'] = pd.qcut(val_df['co2_emissions_intensity_for_consumed_electricity'], 
                                4, ['q1','q2','q3','q4']
                                )

    # Create a leading timestamp
    val_df['next_utc_time'] = val_df['utc_time'].shift(-1)

    # Create quartile dataframe that identifies which datetimes we want populated in chart
    quartile_df=val_df.copy()

    # Create column for previous quartile
    quartile_df['previous_quartile'] = quartile_df['quartile'].shift(1)
    # Create column for next quartile
    quartile_df['next_quartile'] = quartile_df['quartile'].shift(-1)

    # Flag if next quartile is different than quartile 
    quartile_df["diff_next_quartile"] = np.where(quartile_df["quartile"] != quartile_df["next_quartile"], 1, 0)
    # Flag if previous quartile is different than quartile
    quartile_df["diff_previous_quartile"] = np.where(quartile_df["quartile"] != quartile_df["previous_quartile"], 1, 0)

    # Create quartile dataframe for rows where different quartiles are flagged
    quartile_df = quartile_df[(quartile_df['diff_next_quartile'] == 1) | (quartile_df['diff_previous_quartile'] == 1)]

    # Create dataframes for first and fourth quartiles
    q1_quartile_df = quartile_df[(quartile_df['quartile'] == 'q1')]
    q4_quartile_df = quartile_df[(quartile_df['quartile'] == 'q4')]

    # Create dataframes for first and fourth quartiles
    q1_df = val_df[val_df['quartile'] == 'q1']
    q4_df = val_df[val_df['quartile'] == 'q4']

    # Make quartile 1 and 4 values lists
    q1_x0_values = q1_df['utc_time'].tolist()
    q1_x1_values = q1_df['next_utc_time'].tolist()

    q4_x0_values = q4_df['utc_time'].tolist()
    q4_x1_values = q4_df['next_utc_time'].tolist()

    avg_q4_intensity=q4_df['co2_emissions_intensity_for_consumed_electricity'].mean()
    # print(f"Avg. 4th Quartile_Intensity = {avg_q4_intensity}")

    avg_q1_intensity=q1_df['co2_emissions_intensity_for_consumed_electricity'].mean()
    # print(f"Avg. 1st Quartile_Intensity = {avg_q1_intensity}")

    reduced_footprint_percentage = (avg_q4_intensity-avg_q1_intensity)/avg_q4_intensity
    # print(f"Over the next 96 hours, if you consuming electricity during low vs. high intensity periods, you can reduce your carbon emissions impact by an average of {reduced_footprint_percentage:.2%}")

    # This gets rerctangle edges
    q1_rect_x_edges = get_timestamp_edges(q1_x0_values)
    q4_rect_x_edges = get_timestamp_edges(q4_x0_values)

    # This creates a list of rectangle shapes associate with each q1 and q4 region
    q1_x_rect_vals, q1_y_rect_vals = get_rectangle_edges(get_timestamp_edges(q1_x0_values))
    q4_x_rect_vals, q4_y_rect_vals = get_rectangle_edges(get_timestamp_edges(q4_x0_values))

    #                                   0                 1                     2                     3         4               
    quartile_intensities.append([avg_q4_intensity, avg_q1_intensity, reduced_footprint_percentage, q1_df, q1_x_rect_vals,
    #                                   5          6          7              8               9                 10 
                                 q1_y_rect_vals, q4_df, q4_x_rect_vals, q4_y_rect_vals, q1_rect_x_edges, q4_rect_x_edges])
    # print(f"Avg. 4th Quartile_Intensity = {avg_q4_intensity}")
    # print(f"Avg. 1st Quartile_Intensity = {avg_q1_intensity}")
    # print(f"Over the next 96 hours, if you consuming electricity during low vs. high intensity periods, you can reduce your carbon emissions impact by an average of {reduced_footprint_percentage:.2%}")

In [19]:
# # Identify each midnight timestamp to indicate new day

# # Creat column for hour
# val_df['utc_hour'] = val_df['utc_time'].dt.hour
# # Copy dataframe
# midnight_df = val_df.copy()
# # Make dataframe with midnight datetimes
# midnight_df = midnight_df[(midnight_df['utc_hour'] == 0)]
# # Create list with datetimes for each new day 
# mid_x_values = midnight_df['utc_time'].tolist()

## Prepare dataframes for energy mix

In [20]:
# Pull in just the energy generation columns
df_share = df.loc['2021-12-25 08:00:00':'2023-01-05 07:00:00', ['local_time', 'offset', 'ng_col', 'ng_ng', 'ng_nuc', 'ng_oil',
                                                                'ng_wat', 'ng_sun', 'ng_wnd', 'ng_oth']]
df_share = df_share.fillna(value=0.0)
df_share = df_share.rename(columns={'ng_col': 'Coal', 'ng_ng': 'Natural Gas', 'ng_nuc': 'Nuclear',
                              'ng_oil': 'Petroleum', 'ng_wat': 'Hydro', 'ng_sun': 'Solar',
                              'ng_wnd': 'Wind', 'ng_oth': 'Other'})
df_share = df_share.reindex(columns=['local_time', 'offset', 'Coal', 'Petroleum', 'Nuclear', 'Solar', 'Wind', 'Hydro', 'Natural Gas', 'Other'])
df_share

,local_time,offset,Coal,Petroleum,Nuclear,Solar,Wind,Hydro,Natural Gas,Other
utc_time,,,,,,,,,,
2021-12-25 08:00:00,2021-12-25 00:00:00,8,413.0,46.0,2263.0,0.0,3058.0,1482.0,8512.0,685.0
2021-12-25 09:00:00,2021-12-25 01:00:00,8,412.0,46.0,2263.0,0.0,3046.0,1412.0,7956.0,656.0
2021-12-25 10:00:00,2021-12-25 02:00:00,8,411.0,46.0,2265.0,0.0,2819.0,1158.0,7357.0,649.0
2021-12-25 11:00:00,2021-12-25 03:00:00,8,413.0,46.0,2265.0,0.0,2446.0,1170.0,7418.0,657.0
2021-12-25 12:00:00,2021-12-25 04:00:00,8,412.0,46.0,2262.0,0.0,2390.0,1177.0,7254.0,655.0
...,...,...,...,...,...,...,...,...,...,...
2023-01-04 19:00:00,2023-01-04 11:00:00,8,1221.0,85.0,2247.0,5294.0,1493.0,2839.0,12672.0,714.0
2023-01-04 20:00:00,2023-01-04 12:00:00,8,1219.0,85.0,2249.0,5465.0,1569.0,2640.0,12522.0,715.0
2023-01-04 21:00:00,2023-01-04 13:00:00,8,1220.0,85.0,2248.0,4881.0,1552.0,2282.0,12983.0,760.0


In [21]:
# Create a percentage share dataframe
df_local_time = df_share['local_time']
df_offset = df_share['offset']
row_totals = df_share.drop(['local_time', 'offset'], axis=1).sum(axis=1)
df_percent = df_share.drop(['local_time', 'offset'], axis=1).div(row_totals, axis=0)*100 # changed to * 100
df_percent.insert(0, 'local_time', df_local_time)
df_percent.insert(1, 'offset', df_offset)
df_percent

,local_time,offset,Coal,Petroleum,Nuclear,Solar,Wind,Hydro,Natural Gas,Other
utc_time,,,,,,,,,,
2021-12-25 08:00:00,2021-12-25 00:00:00,8,2.509265,0.279482,13.749316,0.000000,18.579501,9.004192,51.716386,4.161857
2021-12-25 09:00:00,2021-12-25 01:00:00,8,2.609081,0.291305,14.330948,0.000000,19.289469,8.941802,50.383130,4.154265
2021-12-25 10:00:00,2021-12-25 02:00:00,8,2.794968,0.312819,15.402924,0.000000,19.170350,7.874872,50.030602,4.413465
2021-12-25 11:00:00,2021-12-25 03:00:00,8,2.865071,0.319112,15.712799,0.000000,16.968436,8.116545,51.460284,4.557752
2021-12-25 12:00:00,2021-12-25 04:00:00,8,2.902226,0.324035,15.934066,0.000000,16.835728,8.291068,51.098901,4.613976
...,...,...,...,...,...,...,...,...,...,...
2023-01-04 19:00:00,2023-01-04 11:00:00,8,4.596273,0.319970,8.458498,19.928477,5.620177,10.686994,47.701863,2.687747
2023-01-04 20:00:00,2023-01-04 12:00:00,8,4.606258,0.321191,8.498337,20.650695,5.928809,9.975816,47.317110,2.701784
2023-01-04 21:00:00,2023-01-04 13:00:00,8,4.690323,0.326785,8.642497,18.765138,5.966706,8.773211,49.913498,2.921841


In [22]:
# This function takes a forecast date, and selects the data for a window around it.
# Each window will be used for plotting one day of the data slider

def update_stack_plot(forecast_date):
    left_window=forecast_date - timedelta(hours=leading_window)
    right_window=forecast_date + timedelta(hours=96) 
    forecast_start = forecast_date + timedelta(hours=1)
    df_day = df_percent.loc[left_window:right_window, ['local_time', 'offset', 'Coal', 'Petroleum',
              'Nuclear', 'Solar', 'Wind',
              'Hydro', 'Natural Gas', 'Other']]
    return df_day

In [23]:
# Until memory issues resolved, select only 100 steps of the year for now
subsample=len(forecasts_df)
# subsample=100

list_of_source_dfs = []

for f in forecasts_df.iloc[0 : subsample].index:
    source_day_df = update_stack_plot(f)
    source_day_df = source_day_df.astype({c: np.float32 for c in source_day_df.select_dtypes(include='float64').columns})
    list_of_source_dfs.append(source_day_df)
print(f"Number of separate dfs (separate timesteps on the plot): {len(list_of_source_dfs)}")

Number of separate dfs (separate timesteps on the plot): 365


# Graph 1 = Actual Carbon Intensity and Forecast Slider with % Footprint Reduction-not used in final presentation

In [24]:
# %%time
# # This is the main graph window

# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data = [], [], []
# initial_position = 182 # this is July 1st

# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         visible=False, )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         visible=False, )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.2, 0.8],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

# # Show the traces from the pair with index[initial position] when screen first loads
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# data = actual_data + forecast_data + horizon_data

# steps = []
# """
# actual_data, forecast_data hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# actual_visible = [False, False, True, False,...]
# forecast_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     # then make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)

# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         # font=dict(size=30),
#                         # color="white",
#                         # len=0.75,
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True)

# fig1 = go.Figure(data=data, layout=layout)
# fig1.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     tickformat="%-m/%-d\n%Y")
# )
# fig1.update_layout(
#     sliders=sliders,
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                   +"would reduce your carbon emissions by<br><br><br>"
#                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#            'x': 0.46
#           },
#     width=1000,
#     height=1000
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )

# # Either show the plot or save it as an offline plot
# fig1.show()
# # offline.plot(fig, include_plotlyjs="cdn", filename="test_carbon_intensity_slider_plot.html")

In [25]:
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/test_carbon_intensity_slider_plot.html'

# # Write chart to html
# fig1.write_html(file_name, auto_play=False)

# Graph 2: Energy Consumption Mix Slider with % Footprint Reduction-not used in final presentation

In [26]:
# %%time
# # This is the main graph function--it can be condensed but for variable naming simplicity
# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []
# horizon_data_2 = []
# initial_position = 182 # this is July 1st

# for d in range(0,len(list_of_source_dfs)):
#     source_day_df = list_of_source_dfs[d]
#     xb = source_day_df.index
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']
#     xfb = source_day_df.index[-96]

#     coal_trace = go.Scatter(
#         x=xb,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         groupnorm='percent',
#         visible=False,)

#     petroleum_trace = go.Scatter(
#         x=xb,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         visible=False, )

#     nuclear_trace = go.Scatter(
#         x=xb,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         visible=False, )
            
#     solar_trace = go.Scatter(
#         x=xb,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         visible=False, )
                
#     wind_trace = go.Scatter(
#         x=xb,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         visible=False, )
                    
#     hydro_trace = go.Scatter(
#         x=xb,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         visible=False, )
                        
#     natural_gas_trace = go.Scatter(
#         x=xb,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         visible=False, )

#     other_trace = go.Scatter(
#         x=xb,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         visible=False, )
    
#     horizon_trace_2 = go.Scatter(
#         x=[xfb, xfb],
#         y=[0, 100],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, )

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)
#     horizon_data_2.append(horizon_trace_2)

# # Show the traces from the pair with index[initial_position] when screen refreshes
# coal_data[initial_position].visible = True
# petroleum_data[initial_position].visible = True
# nuclear_data[initial_position].visible = True
# solar_data[initial_position].visible = True
# wind_data[initial_position].visible = True
# hydro_data[initial_position].visible = True
# natural_gas_data[initial_position].visible = True
# other_data[initial_position].visible = True
# horizon_data_2[initial_position].visible = True

# data = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data + horizon_data_2

# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# for s in range(len(coal_data)):
#     # initially hide all data once
#     coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": coal_visible + petroleum_visible + nuclear_visible + solar_visible + wind_visible + natural_gas_visible + other_visible + horizon_visible_2},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)

# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=40, b=200),
#                         # font=dict(size=30),
#                         # color="white",
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     yaxis=dict(title="Percentage of energy supplied in kWh"),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True
#     )

# fig2 = go.Figure(data=data, layout=layout)
# fig2.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     # dtick="M1",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     tickformat="%-m/%-d\n%Y")
# )
# fig2.update_layout(
#     sliders=sliders,
#     title={'text': "Mix of Electricity Generation on California Grid around<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                   +"would reduce your carbon emissions by<br><br><br>"
#                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#            'x': 0.46
#           },
#     width=1000,
#     height=1000,
#     yaxis=dict(
#         type='linear',
#         range=[0.01,100],
#         ticksuffix='%'
#     )
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )

# # Either show the plot or save it as an offline plot
# fig2.show()
# # offline.plot(fig, include_plotlyjs="cdn", filename="test_energy_mix_slider_plot.html")

In [27]:
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/test_energy_mix_slider_plot.html.html'

# # Write chart to html
# fig2.write_html(file_name, auto_play=False)

# Graph 3: Dual Forecast-Energy Mix Plot on Single Slider-not used in final presentation

In [28]:
# %%time
# # This is the main graph window

# fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.25)
# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data = [], [], []

# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []
# horizon_data_2 = []

# initial_position = 182 # this is July 1st


# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]
    
#     source_day_df = list_of_source_dfs[d]
#     xb = source_day_df.index
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         visible=False, )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         visible=False, )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.2, 0.8],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, )

#     coal_trace = go.Scatter(
#         x=xb,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         groupnorm='percent',
#         visible=False,)

#     petroleum_trace = go.Scatter(
#         x=xb,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         visible=False, )

#     nuclear_trace = go.Scatter(
#         x=xb,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         visible=False, )
            
#     solar_trace = go.Scatter(
#         x=xb,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         visible=False, )
                
#     wind_trace = go.Scatter(
#         x=xb,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         visible=False, )
                    
#     hydro_trace = go.Scatter(
#         x=xb,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         visible=False, )
                        
#     natural_gas_trace = go.Scatter(
#         x=xb,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         visible=False, )

#     other_trace = go.Scatter(
#         x=xb,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         visible=False, )
    
#     horizon_trace_2 = go.Scatter(
#         x=[xf, xf],
#         y=[0, 100],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, )


#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)
#     horizon_data_2.append(horizon_trace_2)

# # Show the traces from the pair with index[initial_position] when screen refreshes
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# coal_data[initial_position].visible = True
# petroleum_data[initial_position].visible = True
# nuclear_data[initial_position].visible = True
# solar_data[initial_position].visible = True
# wind_data[initial_position].visible = True
# hydro_data[initial_position].visible = True
# natural_gas_data[initial_position].visible = True
# other_data[initial_position].visible = True
# horizon_data_2[initial_position].visible = True


# data = actual_data + forecast_data + horizon_data
# data_2 = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data + horizon_data_2


In [29]:
# %%time
# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible + coal_visible + petroleum_visible + \
#                        nuclear_visible + solar_visible + wind_visible + natural_gas_visible + other_visible + horizon_visible_2},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)

In [30]:
# %%time
# ##########################
# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         # font=dict(size=30),
#                         # color="white",
#                         y=0.6,
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True)

In [31]:
# %%time
# fig3=go.Figure()
# fig3=make_subplots(rows=2, 
#                   cols=1, 
#                   shared_xaxes=True,
#                   row_width=[0.4, 0.4],   # Subplot heights
#                   # vertical_spacing=0.2   # Vertical space between subplots
#                  )

In [32]:
# %%time
# fig3.add_traces(data=data, rows=1, cols=1)
# fig3.add_traces(data=data_2, rows=2, cols=1)

# # fig.add_traces(data=data_2, rows=2, cols=1)
# # fig1 = go.Figure(data=data, layout=layout)

# fig3.update_layout(
#     sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.46
#           },
#     width=1000,
#     height=1000,
#     # row=1,
#     # col=1
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )


# # Either show the plot or save it as an offline plot
# # fig3.show()
# # offline.plot(fig3, include_plotlyjs="cdn", filename="carbon_intensity_and_source_two_charts_single_slider_plot.html")

In [33]:
# fig3.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     # dtick="M1",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     # tickformat="%-m/%-d\n%Y", row=1, col=1)
#     )
# )


In [34]:
# offline.plot(fig3, include_plotlyjs="cdn", filename="carbon_intensity_and_source_two_charts_single_slider_plot.html")
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_two_charts_single_slider_plot.html'

# # Write chart to html
# fig3.write_html(file_name, auto_play=False)

In [35]:
# print(this_break_stops_code_from_running_below)

#Graph 4: Carbon Intensity and Energy Mix on a Single Slider Double Y Axis - not used in final presentation

In [36]:
# %%time
# # This is the main graph window

# # fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.25)
# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data = [], [], []

# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []
# # horizon_data_2 = []

# initial_position = 182 # this is July 1st


# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]
    
#     source_day_df = list_of_source_dfs[d]
#     xb = source_day_df.index
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=3),
#         visible=False, 
#         yaxis='y2',
#         )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=3),
#         visible=False, 
#         yaxis='y2',
#         )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.2, 0.8],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, 
#         yaxis='y2',
#         )
    
#     coal_trace = go.Scatter(
#         x=xb,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         groupnorm='percent',
#         visible=False,
#         yaxis='y1',
#         )

#     petroleum_trace = go.Scatter(
#         x=xb,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )

#     nuclear_trace = go.Scatter(
#         x=xb,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )
            
#     solar_trace = go.Scatter(
#         x=xb,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )
                
#     wind_trace = go.Scatter(
#         x=xb,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )
                    
#     hydro_trace = go.Scatter(
#         x=xb,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )
                        
#     natural_gas_trace = go.Scatter(
#         x=xb,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )

#     other_trace = go.Scatter(
#         x=xb,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         )
    
#     # horizon_trace_2 = go.Scatter(
#     #     x=[xf, xf],
#     #     y=[0, 100],
#     #     mode='lines',
#     #     name='Horizon',
#     #     line=dict(color='grey', width=3, dash='dash'),
#     #     visible=False,
#     #     yaxis='y2',
#     #     )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)
#     # horizon_data_2.append(horizon_trace_2)

# # Show the traces from the pair with index[initial_position] when screen refreshes
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# coal_data[initial_position].visible = True
# petroleum_data[initial_position].visible = True
# nuclear_data[initial_position].visible = True
# solar_data[initial_position].visible = True
# wind_data[initial_position].visible = True
# hydro_data[initial_position].visible = True
# natural_gas_data[initial_position].visible = True
# other_data[initial_position].visible = True
# # horizon_data_2[initial_position].visible = True

# # Original Order
# data = actual_data + forecast_data + horizon_data
# data_2 = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data # + horizon_data_2

# # Reverse Order
# # data = horizon_visible + forecast_visible + actual_visible
# # data_2 = other_visible + natural_gas_visible + hydro_data + wind_visible + solar_visible + nuclear_visible + petroleum_visible + coal_visible


In [37]:
# %%time
# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     # horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     # horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": coal_visible + petroleum_visible + \
#                        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
#                        actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   # +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   # +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   # +"would reduce your carbon emissions by<br><br><br>"
#                                   # +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                       },
#                       ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)
#                 # args=[{"visible": actual_visible + forecast_visible + horizon_visible + coal_visible + petroleum_visible + \
#                 #        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible},
#                 # args=[{"visible": other_visible + natural_gas_visible + hydro_visible + wind_visible + solar_visible + nuclear_visible + \
#                 #       petroleum_visible + coal_visible + horizon_visible + forecast_visible + actual_visible},

In [38]:
                # args=[{"visible": actual_visible + forecast_visible + horizon_visible},
                #       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
                      
                #       #  coal_visible + petroleum_visible + \
                #       #  nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
                       
                #                   # +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
                #                   # +"Consuming electricity at the lows vs. highs in this forecast window<br>"
                #                   # +"would reduce your carbon emissions by<br><br><br>"
                #                   # +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
                #                   },
                #       ],
                # label=forecasts_df.index[s].strftime('%b<br> %Y '),
                # )

In [39]:
# %%time
# ##########################
# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         # pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         pad=dict(t=40),
#                         # font=dict(size=30),
#                         # color="white",
#                         # y=0.6,
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)",
    
#     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", overlaying="y", side="right", position=0.15),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True)

In [40]:
# %%time
# fig4=go.Figure()
# # fig4=make_subplots(rows=2, 
# #                   cols=1, 
# #                   shared_xaxes=True,
# #                   row_width=[0.4, 0.4],   # Subplot heights
# #                   # vertical_spacing=0.2   # Vertical space between subplots
# #                  )

In [41]:
# %%time


# # Original order
# # fig4.add_traces(data=data)
# # fig4.add_traces(data=data_2)

# # Reverse order
# fig4.add_traces(data=data_2)
# fig4.add_traces(data=data)



# # fig4.add_traces(data=data, rows=1, cols=1)
# # fig4.add_traces(data=data_2, rows=2, cols=1)

# # fig.add_traces(data=data_2, rows=2, cols=1)
# # fig1 = go.Figure(data=data, layout=layout)

# fig4.update_layout(
#     sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.46
#           },
#     width=1000,
#     height=700,
#     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
#     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#     #             range=[0.01,100],ticksuffix='%',overlaying="y", side="right"),
#     yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#                 range=[0.01,100],ticksuffix='%', side="right"),
#     yaxis2=dict(title="CO2 Intensity (lbs CO2/kWh)", overlaying="y"),
#     margin=dict(r=100),
#     legend=dict(x=1.1, y=0.5),
#     # row=1,
#     # col=1
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )


# # Either show the plot or save it as an offline plot
# # fig4.show()
# # offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")

In [42]:
# offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_double_axis_single_slider_plot.html'

# # Write chart to html
# fig4.write_html(file_name, auto_play=False)

In [43]:
# fig4.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     # dtick="M1",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     # tickformat="%-m/%-d\n%Y", row=1, col=1)
#     )
# )


#Graph 4b: Trying Graph 4 with Scattergl (no improvement in speed, not used in final presentation)

In [44]:
# %%time
# # This is the main graph window

# # fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.25)
# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data = [], [], []

# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []
# # horizon_data_2 = []

# initial_position = 182 # this is July 1st


# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]
    
#     source_day_df = list_of_source_dfs[d]
#     xb = source_day_df.index
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']

#     actual_trace = go.Scattergl(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=3),
#         visible=False, 
#         yaxis='y2',
#         )

#     forecast_trace = go.Scattergl(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=3),
#         visible=False, 
#         yaxis='y2',
#         )
    
#     horizon_trace = go.Scattergl(
#         x=[xf, xf],
#         y=[0.2, 0.8],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, 
#         yaxis='y2',
#         )
    
#     # coal_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb1,
#     #     mode='lines',
#     #     name='Coal',
#     #     line=dict(color='#84501d', width=0.5),
#     #     stackgroup='one',
#     #     groupnorm='percent',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )

#     # petroleum_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb2,
#     #     mode='lines',
#     #     name='Petroleum',
#     #     line=dict(color='#7d6c9a', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )

#     # nuclear_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb3,
#     #     mode='lines',
#     #     name='Nuclear',
#     #     line=dict(color='#a33240', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )
            
#     # solar_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb4,
#     #     mode='lines',
#     #     name='Solar',
#     #     line=dict(color='#ffc602', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )
                
#     # wind_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb5,
#     #     mode='lines',
#     #     name='Wind',
#     #     line=dict(color='#5d9732', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )
                    
#     # hydro_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb6,
#     #     mode='lines',
#     #     name='Hydro',
#     #     line=dict(color='#72c5e8', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )
                        
#     # natural_gas_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb7,
#     #     mode='lines',
#     #     name='Natural Gas',
#     #     line=dict(color='#dbb18a', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )

#     # other_trace = go.Scattergl(
#     #     x=xb,
#     #     y=yb8,
#     #     mode='lines',
#     #     name='Other',
#     #     line=dict(color='#adadad', width=0.5),
#     #     stackgroup='one',
#     #     visible=False,
#     #     yaxis='y1',
#     #     )
    
#     # horizon_trace_2 = go.Scatter(
#     #     x=[xf, xf],
#     #     y=[0, 100],
#     #     mode='lines',
#     #     name='Horizon',
#     #     line=dict(color='grey', width=3, dash='dash'),
#     #     visible=False,
#     #     yaxis='y2',
#     #     )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     # coal_data.append(coal_trace)
#     # petroleum_data.append(petroleum_trace)
#     # nuclear_data.append(nuclear_trace)
#     # solar_data.append(solar_trace)
#     # wind_data.append(wind_trace)
#     # hydro_data.append(hydro_trace)
#     # natural_gas_data.append(natural_gas_trace)
#     # other_data.append(other_trace)
#     # # horizon_data_2.append(horizon_trace_2)

# # Show the traces from the pair with index[initial_position] when screen refreshes
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# # coal_data[initial_position].visible = True
# # petroleum_data[initial_position].visible = True
# # nuclear_data[initial_position].visible = True
# # solar_data[initial_position].visible = True
# # wind_data[initial_position].visible = True
# # hydro_data[initial_position].visible = True
# # natural_gas_data[initial_position].visible = True
# # other_data[initial_position].visible = True
# # # horizon_data_2[initial_position].visible = True

# # Original Order
# data = actual_data + forecast_data + horizon_data
# # data_2 = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data # + horizon_data_2

# # Reverse Order
# # data = horizon_visible + forecast_visible + actual_visible
# # data_2 = other_visible + natural_gas_visible + hydro_data + wind_visible + solar_visible + nuclear_visible + petroleum_visible + coal_visible

In [45]:
# %%time
# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     # coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     # wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     # # horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     # coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     # wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     # # horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
                      
#                       #  coal_visible + petroleum_visible + \
#                       #  nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
                       
#                                   # +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   # +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   # +"would reduce your carbon emissions by<br><br><br>"
#                                   # +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                       ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)
#                 # args=[{"visible": actual_visible + forecast_visible + horizon_visible + coal_visible + petroleum_visible + \
#                 #        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible},
#                 # args=[{"visible": other_visible + natural_gas_visible + hydro_visible + wind_visible + solar_visible + nuclear_visible + \
#                 #       petroleum_visible + coal_visible + horizon_visible + forecast_visible + actual_visible},

In [46]:
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     # then make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)

# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         # font=dict(size=30),
#                         # color="white",
#                         minorticklen=2,
#                         steps=steps)]

In [47]:
# %%time
# ##########################
# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         # pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         pad=dict(t=40),
#                         # font=dict(size=30),
#                         # color="white",
#                         # y=0.6,
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)",
    
#     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", overlaying="y", side="right", position=0.15),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True)

In [48]:
# %%time
# fig4=go.Figure()
# # fig4=make_subplots(rows=2, 
# #                   cols=1, 
# #                   shared_xaxes=True,
# #                   row_width=[0.4, 0.4],   # Subplot heights
# #                   # vertical_spacing=0.2   # Vertical space between subplots
# #                  )

In [49]:
# %%time


# # Original order
# # fig4.add_traces(data=data)
# # fig4.add_traces(data=data_2)

# # Reverse order
# fig4.add_traces(data=data_2)
# fig4.add_traces(data=data)



# # fig4.add_traces(data=data, rows=1, cols=1)
# # fig4.add_traces(data=data_2, rows=2, cols=1)

# # fig.add_traces(data=data_2, rows=2, cols=1)
# # fig1 = go.Figure(data=data, layout=layout)

# fig4.update_layout(
#     sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.46
#           },
#     width=1000,
#     height=700,
#     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
#     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#     #             range=[0.01,100],ticksuffix='%',overlaying="y", side="right"),
#     yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#                 range=[0.01,100],ticksuffix='%', side="right"),
#     yaxis2=dict(title="CO2 Intensity (lbs CO2/kWh)", overlaying="y"),
#     margin=dict(r=100),
#     legend=dict(x=1.1, y=0.5),
#     # row=1,
#     # col=1
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )


# # Either show the plot or save it as an offline plot
# # fig4.show()
# # offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")

In [50]:
# offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_double_axis_single_slider_plot.html'

# # Write chart to html
# fig4.write_html(file_name, auto_play=False)

In [51]:
# fig4.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     # dtick="M1",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     # tickformat="%-m/%-d\n%Y", row=1, col=1)
#     )
# )


#Graph 4c: A refactored version of 4a and 4b - not used in final presentation

In [52]:
# %%time
# # This is the main graph window

# actual_data, forecast_data, horizon_data = [], [], []
# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []

# initial_position = 182 # this is July 1st

# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]
    
#     source_day_df = list_of_source_dfs[d]
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         visible=False, 
#         yaxis='y2',
#         legendgroup='group',    # Group legend
#         legendgrouptitle_text='Carbon Intensity',   # Name legend group
#         hovertemplate='%{y:.2f}',
#         )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.15, 0.82],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='#515151', width=2, dash='dot'),
#         visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     coal_trace = go.Scatter(
#         x=x,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         groupnorm='percent',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         legendgrouptitle_text='Energy Source',
#         hovertemplate='%{y:.2f}%',
#         )

#     petroleum_trace = go.Scatter(
#         x=x,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     nuclear_trace = go.Scatter(
#         x=x,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
            
#     solar_trace = go.Scatter(
#         x=x,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                
#     wind_trace = go.Scatter(
#         x=x,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                    
#     hydro_trace = go.Scatter(
#         x=x,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                        
#     natural_gas_trace = go.Scatter(
#         x=x,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     other_trace = go.Scatter(
#         x=x,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)

# # Show the traces from the pair with index[initial_position] when screen refreshes
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# coal_data[initial_position].visible = True
# petroleum_data[initial_position].visible = True
# nuclear_data[initial_position].visible = True
# solar_data[initial_position].visible = True
# wind_data[initial_position].visible = True
# hydro_data[initial_position].visible = True
# natural_gas_data[initial_position].visible = True
# other_data[initial_position].visible = True
# # horizon_data_2[initial_position].visible = True

# # Original Order
# # data = actual_data + forecast_data + horizon_data
# # data_2 = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data # + horizon_data_2

# # Reverse Order
# # data = horizon_data + forecast_data + actual_data
# # data_2 = other_data + natural_gas_data + hydro_data + wind_data + solar_data + nuclear_data + petroleum_data + coal_data

# # Hybrid Order: forecasts reversed, energy mix selected to match layers on chart
# data = horizon_data + forecast_data + actual_data
# data_2 = coal_data + petroleum_data + nuclear_data + other_data + solar_data + wind_data + hydro_data + natural_gas_data

In [53]:
# %%time
# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     # horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     # horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": coal_visible + petroleum_visible + \
#                        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
#                        actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   # +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   # +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   # +"would reduce your carbon emissions by<br><br><br>"
#                                   # +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                       },
#                       ],
#                 label=forecasts_df.index[s].strftime('    %b    <br> %Y '),
#                 )
#     steps.append(step)
#                 # args=[{"visible": actual_visible + forecast_visible + horizon_visible + coal_visible + petroleum_visible + \
#                 #        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible},
#                 # args=[{"visible": other_visible + natural_gas_visible + hydro_visible + wind_visible + solar_visible + nuclear_visible + \
#                 #       petroleum_visible + coal_visible + horizon_visible + forecast_visible + actual_visible},


# ##########################
# sliders = [dict(active=initial_position,
#                 currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                 # pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                 pad=dict(t=40),
#                 # font=dict(size=30),
#                 # color="white",
#                 # y=0.6,
#                 minorticklen=2,
#                 steps=steps)]

# # layout = go.Layout(
# #     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
# #     # xaxis=dict(title="x"),
# #     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)",
# #     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", overlaying="y", side="right", position=0.15),
# #     xaxis=dict(showgrid=False),
# #     yaxis=dict(showgrid=False),
# #     font=dict(size=16),
# #     hovermode='x unified',
# #     hoverlabel=dict(font_size=16),
# #     sliders=sliders,
# #     showlegend=True,
# #     # plot_bgcolor='rgba(0,0,0,0)',
# #     # legend={'traceorder':'reversed+grouped'},
# #     )

In [54]:
# %%time
# fig4=go.Figure()

# # Reverse order
# fig4.add_traces(data=data_2)
# fig4.add_traces(data=data)

# fig4.update_layout(
#     sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.42
#           },
#     width=1000,
#     height=700,
#     yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#                 range=[0.01,100],tickprefix=' ',ticksuffix='%', side="right", showgrid=False),
#     yaxis2=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)", range=[0.15,0.82], ticksuffix=' ', overlaying="y", showgrid=False),
#     margin=dict(r=100),
#     legend=dict(x=1.1, y=0.5),
#     plot_bgcolor='rgba(0, 0, 0, 0)'
# )
# # fig4.update_yaxes(secondary_y=False, range=[0.15, 0.85])



# # Either show the plot or save it as an offline plot
# fig4.show()


In [55]:
# offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_double_axis_single_slider_plot.html'

# # Write chart to html
# fig4.write_html(file_name, auto_play=False)

#Graph 4d [IN FINAL PRESENTATION]: Converting to local time

In [56]:
bronte_colors = {
    'horizon': '#515151', # darker grey
    'forecast': "#4e8538",  # blue
    'actual': '#152404',  
    'coal': '#303036',  # dark grey
    'petroleum': '#152404', # black (greenish black)
    'nuclear': '#a33240', # maroonish red
    'solar': '#FFB30F',   # yellow
    'wind': '#4E8538', # green
    'hydro': '#437F97', # blue
    'natural_gas': '#dbb18a', # beige
    'other': '#3A5867', # light grey 
}

# light-green: #4e8538;
# dark-green: #396129;

hex_colors = {
    'horizon': '#515151', # darker grey
    'forecast': "blue",  # blue
    'actual': 'black', 
    'coal': '#303036',  # dark grey
    'petroleum': '#152404', # black (greenish black)
    'nuclear': '#a33240', # maroonish red
    'solar': '#FFB30F',   # yellow
    'wind': '#4E8538', # green
    'hydro': '#437F97', # blue
    'natural_gas': '#dbb18a', # beige
    'other': '#3A5867', # light grey 
}
old_colors={
    'horizon': '#515151', # darker grey
    'forecast': "blue",  # blue
    'actual': 'black', # black
    'coal': '#84501d', # tannish brown
    'petroleum': '#7d6c9a', # light grey
    'nuclear': '#a33240', # maroonish red
    'solar': '#ffc602', # golden yellow
    'wind': '#5d9732', # green
    'hydro': '#72c5e8', # light blue
    'natural_gas': '#dbb18a', # beige
    'other': '#adadad', # light grey
}

palette=bronte_colors

In [57]:
%%time
# This is the main graph window

actual_data, forecast_data, horizon_data = [], [], []
coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []

initial_position = 208 # this is July 1st

# Step through each daily dataframe (1 window corresponding to 1 slider step)
# Add individual trades to the slider step graph for each day
for d in range(0,len(list_of_dfs)):
    carbon_day_df = list_of_dfs[d]
    x = carbon_day_df['local_time']
    y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
    y2 = carbon_day_df['co2_forecast']
    xf = carbon_day_df.index[-96] - pd.Timedelta(hours=carbon_day_df.iloc[-96]['offset']) #96 hours minus 7 or 8 for local time adjustment
    
    source_day_df = list_of_source_dfs[d]
    yb1 = source_day_df['Coal']
    yb2 = source_day_df['Petroleum']
    yb3 = source_day_df['Nuclear']
    yb4 = source_day_df['Solar']
    yb5 = source_day_df['Wind']
    yb6 = source_day_df['Hydro']
    yb7 = source_day_df['Natural Gas']
    yb8 = source_day_df['Other']


## CO2 INTENSITY ##

    actual_trace = go.Scatter(
        x=x,
        y=y,
        name="Actual",
        line=dict(color=palette['actual'], width=2),
        visible=False, 
        yaxis='y2',
        legendgroup='group',    # Group legend
        legendgrouptitle_text='Carbon Intensity',   # Name legend group
        hovertemplate='%{y:.2f}',
        )

    forecast_trace = go.Scatter(
        x=x,
        y=y2,
        name="Forecast",
        line=dict(color=palette['forecast'], width=2),
        visible=False, 
        yaxis='y2',
        legendgroup='group',
        hovertemplate='%{y:.2f}',
        )
    
    horizon_trace = go.Scatter(
        x=[xf, xf],
        y=[0.15, 0.82],
        mode='lines',
        name='Horizon',
        line=dict(color=palette['horizon'], width=2, dash='dot'),
        visible=False, 
        yaxis='y2',
        legendgroup='group1',    # Group legend
        legendgrouptitle_text='Forecast Start',   # Name legend group
        )

 ## ENERGY SOURCES ## 

    coal_trace = go.Scatter(
        x=x,
        y=yb1,
        mode='lines',
        name='Coal',
        line=dict(color=palette['coal'], width=0.5),
        stackgroup='one',
        # groupnorm='percent',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        legendgrouptitle_text='Energy Source',
        hovertemplate='%{y:.2f}%',
        )

    petroleum_trace = go.Scatter(
        x=x,
        y=yb2,
        mode='lines',
        name='Petroleum',
        line=dict(color=palette['petroleum'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )

    nuclear_trace = go.Scatter(
        x=x,
        y=yb3,
        mode='lines',
        name='Nuclear',
        line=dict(color=palette['nuclear'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )
            
    solar_trace = go.Scatter(
        x=x,
        y=yb4,
        mode='lines',
        name='Solar',
        line=dict(color=palette['solar'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )
                
    wind_trace = go.Scatter(
        x=x,
        y=yb5,
        mode='lines',
        name='Wind',
        line=dict(color=palette['wind'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )
                    
    hydro_trace = go.Scatter(
        x=x,
        y=yb6,
        mode='lines',
        name='Hydro',
        line=dict(color=palette['hydro'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )
                        
    natural_gas_trace = go.Scatter(
        x=x,
        y=yb7,
        mode='lines',
        name='Natural Gas',
        line=dict(color=palette['natural_gas'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )

    other_trace = go.Scatter(
        x=x,
        y=yb8,
        mode='lines',
        name='Other',
        line=dict(color=palette['other'], width=0.5),
        stackgroup='one',
        visible=False,
        yaxis='y1',
        legendgroup='group2',
        hovertemplate='%{y:.2f}%',
        )

    actual_data.append(actual_trace)
    forecast_data.append(forecast_trace)
    horizon_data.append(horizon_trace)

    coal_data.append(coal_trace)
    petroleum_data.append(petroleum_trace)
    nuclear_data.append(nuclear_trace)
    solar_data.append(solar_trace)
    wind_data.append(wind_trace)
    hydro_data.append(hydro_trace)
    natural_gas_data.append(natural_gas_trace)
    other_data.append(other_trace)

# Show the traces from the pair with index[initial_position] when screen refreshes
actual_data[initial_position].visible = True
forecast_data[initial_position].visible = True
horizon_data[initial_position].visible = True

coal_data[initial_position].visible = True
petroleum_data[initial_position].visible = True
nuclear_data[initial_position].visible = True
solar_data[initial_position].visible = True
wind_data[initial_position].visible = True
hydro_data[initial_position].visible = True
natural_gas_data[initial_position].visible = True
other_data[initial_position].visible = True

data = actual_data + forecast_data + horizon_data
data_2 = coal_data + petroleum_data + nuclear_data + other_data + solar_data + wind_data + hydro_data + natural_gas_data

CPU times: user 2.19 s, sys: 79.2 ms, total: 2.27 s
Wall time: 2.19 s


In [58]:
%%time
steps = []
"""
coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
For example, when the slider is 2,
coal_visible = [False, False, True, False,...]
petroleum_visible = [False, False, True, False,...]
By combining the two afterward, 2 traces can be displayed at once
"""
# Create the actual graph object for each day
for s in range(len(actual_data)):
    # initially hide all data once
    actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
    coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
    wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)

    #Make only the sth traces visible
    actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
    coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
    wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True

    step = dict(method="update",
                args=[{"visible": coal_visible + petroleum_visible + \
                       nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
                       actual_visible + forecast_visible + horizon_visible},
                      {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')},
                      ],
                label=forecasts_df.index[s].strftime('  %b   <br>    %Y   '),
                )
    steps.append(step)

sliders = [dict(active=initial_position,
                currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
                # pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
                pad=dict(t=70),
                # font=dict(size=30),
                # color="white",
                # y=0.6,
                minorticklen=2,
                steps=steps)]


CPU times: user 62.1 ms, sys: 3.05 ms, total: 65.1 ms
Wall time: 64 ms


In [59]:
%%time
fig4d=go.Figure()

# Add in order
fig4d.add_traces(data=data)
fig4d.add_traces(data=data_2)

fig4d.update_layout(
    sliders=sliders,
    title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
           'x': 0.43,
           'font': {'size': 24}
          },
    width=1100,
    height=620,
    yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source",
                type='linear',
                range=[0.01,100],
                tickprefix=' ',
                ticksuffix='%',
                side="right",
                showgrid=False),
    yaxis2=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)",
                range=[0.15,0.82],
                ticksuffix=' ',
                overlaying="y",
                showgrid=False),
    xaxis=dict(title=dict(font=dict(size=15))),
    margin=dict(r=100),
    legend=dict(x=1.14,
                y=0.5,
                groupclick='toggleitem', 
                font=dict(size=15),
                grouptitlefont=dict(size=15)
                ),
                
    legend_title=dict(font=dict(size=15)),
    plot_bgcolor='white',
    hovermode='x unified',
)
fig4d.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey", tickfont=dict(size=15),
                   title_text="Local Date and Time")
fig4d.update_yaxes(showgrid=False, tickfont=dict(size=13))

# Either show the plot or save it as an offline plot
fig4d.show()

Output hidden; open in https://colab.research.google.com to view.

In [60]:
# offline.plot(fig4d, include_plotlyjs="cdn", filename="website_slider_viz.html")
# # # Save chart as html

# # # Set file name for saved chart
# file_name = 'Data Visualizations/website_slider_viz.html'

# # # Write chart to html
# fig4d.write_html(file_name, auto_play=False)


# offline.plot(fig4d, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")
# # # Save chart as html

# # # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_double_axis_single_slider_plot.html'

# # # Write chart to html
# fig4d.write_html(file_name, auto_play=False)

#Graph 4e: Speed Slider - testing jquery image slider to increase speed, not used in final presentation





In [61]:
# %%time
# # This is the main graph window

# # initial_position = 182 # this is July 1st
# fig4e=go.Figure()

# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
# # for d in range(0,3):
#     actual_data, forecast_data, horizon_data = [], [], []
#     coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
#     wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []
    
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df['local_time']
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     # xf = carbon_day_df.index[-96] #96 hours plus 8 for local time adjustment
#     xf = carbon_day_df.index[-96] - pd.Timedelta(hours=carbon_day_df.iloc[-96]['offset']) #96 hours plus 8 for local time adjustment
    
#     source_day_df = list_of_source_dfs[d]
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']
    
#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',    # Group legend
#         legendgrouptitle_text='Carbon Intensity',   # Name legend group
#         hovertemplate='%{y:.2f}',
#         )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.15, 0.82],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='#515151', width=2, dash='dot'),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     coal_trace = go.Scatter(
#         x=x,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         # groupnorm='percent',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         legendgrouptitle_text='Energy Source',
#         hovertemplate='%{y:.2f}%',
#         )

#     petroleum_trace = go.Scatter(
#         x=x,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     nuclear_trace = go.Scatter(
#         x=x,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
            
#     solar_trace = go.Scatter(
#         x=x,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                
#     wind_trace = go.Scatter(
#         x=x,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                    
#     hydro_trace = go.Scatter(
#         x=x,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                        
#     natural_gas_trace = go.Scatter(
#         x=x,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     other_trace = go.Scatter(
#         x=x,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)

#     # Revised Order
#     data = actual_data + forecast_data + horizon_data
#     data_2 = coal_data + petroleum_data + nuclear_data + other_data + solar_data + wind_data + hydro_data + natural_gas_data
    
#     fig4e.data = []
#     fig4e.layout = {}
#     fig4e.add_traces(data=data)
#     fig4e.add_traces(data=data_2)


#     fig4e.update_layout(
#         # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#         #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#         #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#         #               +"would reduce your carbon emissions by<br><br><br>"
#         #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#         title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#               'x': 0.43,
#               'font': {'size': 24}
#               },
#         # xaxis_tickformat = '%B %-d %p',
#         # xaxis_tickformat = '%d %b<br>%Y<br>%I%p',%B %d, %Y
#         width=1200,
#         height=700,
#         yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source",
#                     type='linear',
#                     # range=[0.01,100],
#                     range=[0.0001,1],
#                     tickprefix=' ',
#                     tickformat=',.0%',
#                     # ticksuffix='%',
#                     side="right",
#                     showgrid=False),
#         yaxis2=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)",
#                     range=[0.15,0.82],
#                     ticksuffix=' ',
#                     overlaying="y",
#                     showgrid=False),
#         xaxis=dict(title=dict(font=dict(size=15))),
#         margin=dict(r=100),
#         legend=dict(x=1.14,
#                     y=0.5,
#                     groupclick='toggleitem', 
#                     font=dict(size=15),
#                     grouptitlefont=dict(size=15)
#                     ),
                    
#         legend_title=dict(font=dict(size=15)),
#         # legendgrouptitle=dict(font=dict(size=15)),
#         plot_bgcolor='white',
#         hovermode='x unified',
#     )
#     fig4e.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey", tickfont=dict(size=15),
#                       title_text="Local Date and Time")
#     fig4e.update_yaxes(showgrid=False, tickfont=dict(size=13))

#     # offline.plot(fig4e, include_plotlyjs="cdn", filename=f"double_axis_plot_{d}.html")
    
#     # Set file name for saved chart
#     # file_name = f'Data Visualizations/Energy_Mix_images/double_axis_plot_{d}.html'

#     # Write chart to html
#     fig4e.write_image(f'Data Visualizations/Energy_Mix_Images/double_axis_plot_{d}.png')


# # Either show the plot or save it as an offline plot
# fig4e.show()    

In [62]:
# %%time
# # This is the main graph window

# actual_data, forecast_data, horizon_data = [], [], []
# coal_data, petroleum_data, nuclear_data, solar_data = [], [], [], []
# wind_data, hydro_data, natural_gas_data, other_data = [], [], [], []

# # initial_position = 182 # this is July 1st
# fig4e=go.Figure()
# fig4e.data = []
# fig4e.layout = {}
# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# # for d in range(0,len(list_of_dfs)):
# for d in range(0,3):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df['local_time']
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     # xf = carbon_day_df.index[-96] #96 hours plus 8 for local time adjustment
#     xf = carbon_day_df.index[-96] - pd.Timedelta(hours=carbon_day_df.iloc[-96]['offset']) #96 hours plus 8 for local time adjustment
    
#     source_day_df = list_of_source_dfs[d]
#     yb1 = source_day_df['Coal']
#     yb2 = source_day_df['Petroleum']
#     yb3 = source_day_df['Nuclear']
#     yb4 = source_day_df['Solar']
#     yb5 = source_day_df['Wind']
#     yb6 = source_day_df['Hydro']
#     yb7 = source_day_df['Natural Gas']
#     yb8 = source_day_df['Other']

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',    # Group legend
#         legendgrouptitle_text='Carbon Intensity',   # Name legend group
#         hovertemplate='%{y:.2f}',
#         )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.15, 0.82],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='#515151', width=2, dash='dot'),
#         # visible=False, 
#         yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     coal_trace = go.Scatter(
#         x=x,
#         y=yb1,
#         mode='lines',
#         name='Coal',
#         line=dict(color='#84501d', width=0.5),
#         stackgroup='one',
#         groupnorm='percent',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         legendgrouptitle_text='Energy Source',
#         hovertemplate='%{y:.2f}%',
#         )

#     petroleum_trace = go.Scatter(
#         x=x,
#         y=yb2,
#         mode='lines',
#         name='Petroleum',
#         line=dict(color='#7d6c9a', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     nuclear_trace = go.Scatter(
#         x=x,
#         y=yb3,
#         mode='lines',
#         name='Nuclear',
#         line=dict(color='#a33240', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
            
#     solar_trace = go.Scatter(
#         x=x,
#         y=yb4,
#         mode='lines',
#         name='Solar',
#         line=dict(color='#ffc602', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                
#     wind_trace = go.Scatter(
#         x=x,
#         y=yb5,
#         mode='lines',
#         name='Wind',
#         line=dict(color='#5d9732', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                    
#     hydro_trace = go.Scatter(
#         x=x,
#         y=yb6,
#         mode='lines',
#         name='Hydro',
#         line=dict(color='#72c5e8', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )
                        
#     natural_gas_trace = go.Scatter(
#         x=x,
#         y=yb7,
#         mode='lines',
#         name='Natural Gas',
#         line=dict(color='#dbb18a', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     other_trace = go.Scatter(
#         x=x,
#         y=yb8,
#         mode='lines',
#         name='Other',
#         line=dict(color='#adadad', width=0.5),
#         stackgroup='one',
#         # visible=False,
#         yaxis='y1',
#         legendgroup='group2',
#         hovertemplate='%{y:.2f}%',
#         )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

#     coal_data.append(coal_trace)
#     petroleum_data.append(petroleum_trace)
#     nuclear_data.append(nuclear_trace)
#     solar_data.append(solar_trace)
#     wind_data.append(wind_trace)
#     hydro_data.append(hydro_trace)
#     natural_gas_data.append(natural_gas_trace)
#     other_data.append(other_trace)

#     # Hybrid Order: forecasts reversed, energy mix selected to match layers on chart
#     data = horizon_data + forecast_data + actual_data
#     data_2 = coal_data + petroleum_data + nuclear_data + other_data + solar_data + wind_data + hydro_data + natural_gas_data


    
#     fig4e.data = []
#     fig4e.layout = {}
#     fig4e.add_traces(data=data_2)
#     fig4e.add_traces(data=data)

#     # fig4d=make_subplots(specs=[[{"secondary_y":True}]])
#     # Reverse order
#     # fig4d.add_trace(coal_trace, secondary_y=False)
#     # fig4d.add_traces(data=coal_trace)
#     # fig4d.add_traces(data=petroleum_trace)
#     # fig4d.add_traces(data=nuclear_trace)
#     # fig4d.add_traces(data=other_trace)
#     # fig4d.add_traces(data=solar_trace)
#     # fig4d.add_traces(data=wind_trace)
#     # fig4d.add_traces(data=hydro_trace)
#     # fig4d.add_traces(data=natural_gas_trace)
#     # fig4d.add_traces(data=horizon_trace)
#     # fig4d.add_traces(data=forecast_trace)
#     # fig4d.add_traces(data=actual_trace)
#     # fig4d.add_traces(data=horizon_trace, secondary_y=True)
#     # fig4d.add_trace(forecast_trace, secondary_y=True)
#     # fig4d.add_trace(actual_trace, secondary_y=True)

#     fig4e.update_layout(
#       # sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#       title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[d].strftime('%B %-d, %Y'),
#            'x': 0.42
#           },
#       width=1000,
#       height=700,
#       yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#                   range=[0.01,100],tickprefix=' ',ticksuffix='%', side="right", showgrid=False),
#       yaxis2=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)", side='left', range=[0.15,0.82], ticksuffix=' ', overlaying="y", showgrid=False),
#       margin=dict(r=100),
#       legend=dict(x=1.1, y=0.5),
#     )
#     fig4e.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey")

#     # offline.plot(fig4e, include_plotlyjs="cdn", filename=f"double_axis_plot_{d}.html")
    
#     # Set file name for saved chart
#     # file_name = f'Data Visualizations/Energy_Mix_images/double_axis_plot_{d}.html'

#     # Write chart to html
#     fig4e.write_image(f'Data Visualizations/Energy_Mix_Images/double_axis_plot_{d}.png')


# # Either show the plot or save it as an offline plot
# fig4e.show()

# # # Show the traces from the pair with index[initial_position] when screen refreshes
# # actual_data[initial_position].visible = True
# # forecast_data[initial_position].visible = True
# # horizon_data[initial_position].visible = True

# # coal_data[initial_position].visible = True
# # petroleum_data[initial_position].visible = True
# # nuclear_data[initial_position].visible = True
# # solar_data[initial_position].visible = True
# # wind_data[initial_position].visible = True
# # hydro_data[initial_position].visible = True
# # natural_gas_data[initial_position].visible = True
# # other_data[initial_position].visible = True
# # horizon_data_2[initial_position].visible = True

# # Original Order
# # data = actual_data + forecast_data + horizon_data
# # data_2 = coal_data + petroleum_data + nuclear_data + solar_data + wind_data + hydro_data + natural_gas_data + other_data # + horizon_data_2

# # Reverse Order
# # data = horizon_data + forecast_data + actual_data
# # data_2 = other_data + natural_gas_data + hydro_data + wind_data + solar_data + nuclear_data + petroleum_data + coal_data


In [63]:
# %%time
# steps = []
# """
# coal_data, petroleum_data, etc. hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# coal_visible = [False, False, True, False,...]
# petroleum_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     coal_visible, petroleum_visible, nuclear_visible, solar_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     wind_visible, hydro_visible, natural_gas_visible, other_visible = [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data), [False] * len(coal_data)
#     # horizon_visible_2 = [False] * len(horizon_data_2)

#     #Make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     coal_visible[s], petroleum_visible[s], nuclear_visible[s], solar_visible[s] = True, True, True, True
#     wind_visible[s], hydro_visible[s], natural_gas_visible[s], other_visible[s] = True, True, True, True
#     # horizon_visible_2[s] = True

#     step = dict(method="update",
#                 args=[{"visible": coal_visible + petroleum_visible + \
#                        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible + \
#                        actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   # +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   # +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   # +"would reduce your carbon emissions by<br><br><br>"
#                                   # +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                       },
#                       ],
#                 label=forecasts_df.index[s].strftime('    %b    <br> %Y '),
#                 )
#     steps.append(step)
#                 # args=[{"visible": actual_visible + forecast_visible + horizon_visible + coal_visible + petroleum_visible + \
#                 #        nuclear_visible + solar_visible + wind_visible + hydro_visible + natural_gas_visible + other_visible},
#                 # args=[{"visible": other_visible + natural_gas_visible + hydro_visible + wind_visible + solar_visible + nuclear_visible + \
#                 #       petroleum_visible + coal_visible + horizon_visible + forecast_visible + actual_visible},

                
# ##########################
# sliders = [dict(active=initial_position,
#                 currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                 # pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                 pad=dict(t=40),
#                 # font=dict(size=30),
#                 # color="white",
#                 # y=0.6,
#                 minorticklen=2,
#                 steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     # yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)",
#     # yaxis2=dict(title="Percentage Mix of CA Electricity Grid by Source", overlaying="y", side="right", position=0.15),
#     xaxis=dict(showgrid=False),
#     yaxis=dict(showgrid=False),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True,
#     # legend={'traceorder':'reversed+grouped'},
#     )

In [64]:
# %%time
# fig4=go.Figure()

# # Reverse order
# fig4.add_traces(data=data_2)
# fig4.add_traces(data=data)

# fig4.update_layout(
#     sliders=sliders,
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#     #               +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#     #               +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#     #               +"would reduce your carbon emissions by<br><br><br>"
#     #               +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.42
#           },
#     width=1000,
#     height=700,
#     yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source", type='linear',
#                 range=[0.01,100],tickprefix=' ',ticksuffix='%', side="right", showgrid=False),
#     yaxis2=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)", range=[0.15,0.82], ticksuffix=' ', overlaying="y", showgrid=False),
#     margin=dict(r=100),
#     legend=dict(x=1.1, y=0.5),
# )
# # fig4.update_yaxes(secondary_y=False, range=[0.15, 0.85])



# # Either show the plot or save it as an offline plot
# fig4.show()


In [65]:
# offline.plot(fig4, include_plotlyjs="cdn", filename="carbon_intensity_and_source_double_axis_single_slider_plot.html")
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/carbon_intensity_and_source_double_axis_single_slider_plot.html'

# # Write chart to html
# fig4.write_html(file_name, auto_play=False)

# Graph 5 = Forecasting window with % Footprint Reduction - testing to replicate first carbon intesity green-red Q1-Q4 quartile highlighting graph in plotly, not used in final presention

In [66]:
# # Read just data with actual carbon intensity
# df_gen = df.loc['2021-12-25 08:00:00':'2023-01-05 07:00:00', ['co2_emissions_intensity_for_consumed_electricity']]
# df_gen.head(5)

In [67]:
# leading_window=2*24

In [68]:
# # This function takes a forecast date, and selects the data for a window around it.
# # Each window will be used for plotting one day of the data slider
# def update_plot(forecast_date):
#     # select carbon intensity from that window
#     left_window=forecast_date - timedelta(hours=leading_window)
#     right_window=forecast_date + timedelta(hours=96) 
#     forecast_start = forecast_date + timedelta(hours=1)
#     df_day = df_gen.loc[left_window:right_window, ['co2_emissions_intensity_for_consumed_electricity']]
    
#     # join the forecast carbon intensity for that window
#     forecasts = forecasts_df.loc[forecast_date].squeeze()
#     datetime_index = pd.date_range(forecast_start, periods=96, freq='1H')
#     forecasts.index = datetime_index
#     df_day['co2_forecast'] = forecasts
#     return df_day

In [69]:
# # Until memory issues resolved, select only 100 steps of the year for now
# subsample=len(forecasts_df)
# # subsample=100

# # Create list of dfs (1 for each day of slider plot)
# list_of_dfs = []

# # Used forecast index here because it goes from 1/1/22-12/31/22 without pesky 00:00:00 suffixes
# # since we are using 24 hour strikes anyway.  You will have 365 dfs, one for each day.
# for f in forecasts_df.iloc[0 : subsample].index:
#     carbon_day_df = update_plot(f)
#     carbon_day_df = carbon_day_df.astype({c: np.float32 for c in carbon_day_df.select_dtypes(include='float64').columns})
#     list_of_dfs.append(carbon_day_df)
# print(f"Number of separate dfs (separate timesteps on the plot): {len(list_of_dfs)}")

In [70]:
# get_timestamp_edges(q1_x0_values)

In [71]:
# q1_metavals = [sublist[9] for sublist in quartile_intensities]

In [72]:
# q1_metavals[:2]

In [73]:
# %%time
# # This is the main graph window

# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data, q1_rectangles_data, q4_rectangles_data = [], [], [], [], []
# initial_position = 182 # this is July 1st
# # initial_position = 0

# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
# # for d in range(0,1):
#     # carbon_day_df = list_of_dfs[d]
#     # x = carbon_day_df.index
#     # y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     # y2 = carbon_day_df['co2_forecast']
#     # xf = carbon_day_df.index[-96]
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df['local_time']
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     # xf = carbon_day_df.index[-96] #96 hours plus 8 for local time adjustment
#     xf = carbon_day_df.index[-96] - pd.Timedelta(hours=carbon_day_df.iloc[-96]['offset']) #96 hours plus 8 for local time adjustment

#     # actual_trace = go.Scatter(
#     #     x=x,
#     #     y=y,
#     #     name="Actual",
#     #     line=dict(color="black", width=2),
#     #     visible=False, )

#     # forecast_trace = go.Scatter(
#     #     x=x,
#     #     y=y2,
#     #     name="Forecast",
#     #     line=dict(color="blue", width=2),
#     #     visible=False, )
    
#     # horizon_trace = go.Scatter(
#     #     x=[xf, xf],
#     #     y=[0.2, 0.8],
#     #     mode='lines',
#     #     name='Horizon',
#     #     line=dict(color='grey', width=3, dash='dash'),
#     #     visible=False, )

# ## CO2 INTENSITY ##

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         visible=False, 
#         # yaxis='y2',
#         legendgroup='group',    # Group legend
#         legendgrouptitle_text='Carbon Intensity',   # Name legend group
#         hovertemplate='%{y:.2f}',
#         )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         visible=False, 
#         # yaxis='y2',
#         legendgroup='group',
#         hovertemplate='%{y:.2f}',
#         )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.15, 0.82],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='#515151', width=2, dash='dot'),
#         visible=False, 
#         # yaxis='y2',
#         legendgroup='group1',    # Group legend
#         legendgrouptitle_text='Forecast Start',   # Name legend group
#         )

#     q1_rectangles_trace = go.Scatter(
#         x = quartile_intensities[d][4],
#         y = quartile_intensities[d][5],
#         name='Charge Now',
#         mode='lines',
#         line_width=0,
#         fill="toself",
#         fillcolor="#6eb12e",
#         opacity=0.3,
#         visible=False,
#         # meta=q1_metavals,
#     )

#     q4_rectangles_trace = go.Scatter(
#         x = quartile_intensities[d][7],
#         y = quartile_intensities[d][8],
#         line_width=0,
#         mode='lines',
#         name='No Charge',
#         fill="toself",
#         fillcolor="red",
#         opacity=0,# set to 0.4 to make like green
#         visible=False,
#     )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)
#     q1_rectangles_data.append(q1_rectangles_trace)
#     q4_rectangles_data.append(q4_rectangles_trace)

# # Show the traces from the pair with index[initial position] when screen first loads
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True
# q1_rectangles_data[initial_position].visible = True
# q4_rectangles_data[initial_position].visible = True

# data = actual_data + forecast_data + horizon_data + q1_rectangles_data + q4_rectangles_data

# steps = []
# """
# actual_data, forecast_data hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# actual_visible = [False, False, True, False,...]
# forecast_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible, q1_rectangles_visible, q4_rectangles_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data), [False] * len(actual_data), [False] * len(actual_data)
#     # then make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s], q1_rectangles_visible[s], q4_rectangles_visible[s] = True, True, True, True, True
#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible + q1_rectangles_visible + q4_rectangles_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('    %b    <br> %Y '),
#                 # meta = quartile_intensities[s][9]
#                 )
#     steps.append(step)

# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=70, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         # font=dict(size=30),
#                         # color="white",
#                         minorticklen=2,
#                         steps=steps)]

# # layout = go.Layout(dict(
# #                       xaxis=dict(showline=True),
# #                       yaxis=dict(showline=True)
# #                     ))
# #     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
# #     # xaxis=dict(title="x"),
# #     yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
# #     font=dict(size=16),
# #     hovermode='x unified',
# #     hoverlabel=dict(font_size=16),
# #     sliders=sliders,
# #     showlegend=True,
# #     plot_bgcolor='rgba(0,0,0,0)')

# fig5=go.Figure()

# # Add in order
# fig5.add_traces(data=data)
# # fig5.add_traces(data=data_2)


# fig5.update_layout(
#     sliders=sliders,
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                   +"would reduce your carbon emissions by<br><br><br>"
#                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#     # title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y'),
#            'x': 0.43,
#            'font': {'size': 24}
#           },
#     # xaxis_tickformat = '%B %-d %p',
#     # xaxis_tickformat = '%d %b<br>%Y<br>%I%p',%B %d, %Y
#     width=1200,
#     height=1000,
#     # yaxis=dict(title="Percentage Mix of CA Electricity Grid by Source",
#     #             type='linear',
#     #             # range=[0.01,100],
#     #             range=[0.0001,1],
#     #             tickprefix=' ',
#     #             tickformat=',.0%',
#     #             # ticksuffix='%',
#     #             side="right",
#     #             showgrid=False),
#     yaxis=dict(title="CO<sub>2</sub> Intensity (lbs CO<sub>2</sub>/kWh)",
#                 range=[0.15,0.82],
#                 ticksuffix=' ',
#                 # overlaying="y",
#                 showgrid=False,
#                 showline=True),
#     xaxis=dict(title=dict(font=dict(size=15)),
#                showline=True),
#     # margin=dict(r=100),
#     legend=dict(x=1.05,
#                 y=0.5,
#                 groupclick='toggleitem', 
#                 font=dict(size=15),
#                 grouptitlefont=dict(size=15)
#                 ),
                
#     legend_title=dict(font=dict(size=15)),
#     # legendgrouptitle=dict(font=dict(size=15)),
#     plot_bgcolor='white',
#     hovermode='x unified',
# )
# fig5.update_xaxes(showgrid=True, showline=True, color='black', gridwidth=1, gridcolor="LightGrey", tickfont=dict(size=15),
#                    title_text="Local Date and Time")
# fig5.update_yaxes(showgrid=True, showline=True, color='black', tickfont=dict(size=13))

# # fig5 = go.Figure(data=data, layout=layout)
# # fig5.update_xaxes(dict(
# #     showgrid=True,
# #     showline=True,
# #     linewidth=2, linecolor='black', gridcolor='Red',
# #     # nticks=12,
# #     # type="date",
# #     # tick0="2022-01-15 00:00:00",
# #     # dtick="M1",
# #     dtick="D1",
# #     # tickformat="%b %d\n%Y"
# #     # tickvals=[5.1, 5.9, 6.3, 7.5]
# #     tickformat="%-m/%-d\n%Y",
# #     ticklabelmode="period")
# # )
# # fig5.update_yaxes(dict(
# #     showgrid=True,
# #     showline=True,
# #     linewidth=2, linecolor='black', gridcolor='Red',
# #     ) 
# # )
# # fig5.update_layout(
# #     sliders=sliders,
# #     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
# #                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
# #                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
# #                   +"would reduce your carbon emissions by<br><br><br>"
# #                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
# #            'x': 0.46
# #           },
# #     width=1200,
# #     height=1000,

# # )
#     # # type="date",
#     # # tick0="2022-01-15 00:00:00",
#     # dtick="D1",
#     # # tickformat="%b %d\n%Y"
#     # tickformat="%-m/%-d\n%Y")
#     #     sliders=[dict(
#     #     tickformat='%B %d, %Y',
#     # )]

# # Either show the plot or save it as an offline plot
# fig5.show()
# # offline.plot(fig, include_plotlyjs="cdn", filename="test_carbon_intensity_slider_plot.html")

In [74]:
# fig = go.Figure()

# # Add scatter trace for line
# fig.add_trace(go.Scatter(x=val_df['utc_time'], 
#                          y=val_df['co2_emissions_intensity_for_consumed_electricity'],
#                          mode="lines",
#                          line=dict(color="LightGrey"),
#                          hovertext=val_df['co2_emissions_intensity_for_consumed_electricity'],
#                          opacity=0.4,
#                          showlegend=True,
#                          name="Carbon Intensity Forecast",
#                          hoverinfo="skip",
#                          hovertemplate=None
#                          )
#              )

# # Add vertical rectangles for q1 (low carbon intensity)
# for x0, x1 in zip(q1_x0_values, q1_x1_values):
#     fig.add_vrect(x0=x0, 
#                   x1=x1, 
#                   line_width=0, 
#                   fillcolor="#6eb12e", 
#                   opacity=0.4
#                  )
    
# # Add vertical rectangles for q4 (high carbon intensity)
# for x0, x1 in zip(q4_x0_values, q4_x1_values):
#     fig.add_vrect(x0=x0, 
#                   x1=x1, 
#                   line_width=0, 
#                   fillcolor="red", 
#                   opacity=0.4
#                  )

# # Add vertical lines at start of each new day
# for x0 in mid_x_values:
#     fig.add_shape(type='line',
#                 x0=x0, 
#                 y0=0, 
#                 x1=x0, 
#                 y1=val_df['co2_emissions_intensity_for_consumed_electricity'].max(),
#                 line=dict(color='black', 
#                           width=3, 
#                           dash='dash')
#                 )

# # update layout
# fig.update_layout(title='When to Charge', 
#                   xaxis_title='UTC Time', 
#                   yaxis_title='Carbon Intensity',
#                   paper_bgcolor='rgba(0,0,0,0)',
#                   plot_bgcolor='rgba(0,0,0,0)',
#                   yaxis_visible=False, 
#                   yaxis_showticklabels=False,
#                   height=400,
#                   width=1400
#                   )

# fig.update_xaxes(tickvals=q1_quartile_df['utc_time'].tolist() + q4_quartile_df['utc_time'].tolist(),
#                  tickformat='%I%p', # set tick format
#                  tickangle=300
#                  )

# # Add text for weekday name and date (month and day)
# for i in range(len(mid_x_values)):
#     fig.add_annotation(x=mid_x_values[i], 
#                        y=.5,
#                        text=str(unique_weekdays[i])+'<br>'+str(unique_dates[i]),
#                        showarrow=False,
#                        xshift=40,
#                        yshift=10
#                       )
    
# # Create dummy markers to label legend    
# fig.add_scatter(x=[None],
#                 y=[None],
#                 mode="markers",
#                 name="Charge",
#                 marker=dict(size=7, 
#                             color="#6eb12e", 
#                             opacity=0.4, 
#                             symbol='circle')
#                )

# fig.add_scatter(x=[None],
#                 y=[None],
#                 mode="markers",
#                 name="Do NOT Charge",
#                 marker=dict(size=7, 
#                             color="red", 
#                             opacity=0.4, 
#                             symbol='circle')
#                )


# # show plot
# fig.show()


In [75]:
# fig = go.Figure()

# # Add scatter trace for line
# fig.add_trace(go.Scatter(x=val_df['utc_time'], 
#                          y=val_df['co2_emissions_intensity_for_consumed_electricity'],
#                          mode="lines",
#                          line=dict(color="LightGrey"),
#                          hovertext=val_df['co2_emissions_intensity_for_consumed_electricity'],
#                          opacity=0.4,
#                          showlegend=True,
#                          name="Carbon Intensity Forecast",
#                          hoverinfo="skip",
#                          hovertemplate=None
#                          )
#              )

# # Add vertical rectangles for q1 (low carbon intensity)
# for x0, x1 in zip(q1_x0_values, q1_x1_values):
#     fig.add_vrect(x0=x0, 
#                   x1=x1, 
#                   line_width=0, 
#                   fillcolor="#6eb12e", 
#                   opacity=0.4
#                  )
    
# # Add vertical rectangles for q4 (high carbon intensity)
# for x0, x1 in zip(q4_x0_values, q4_x1_values):
#     fig.add_vrect(x0=x0, 
#                   x1=x1, 
#                   line_width=0, 
#                   fillcolor="red", 
#                   opacity=0.4
#                  )

# # Add vertical lines at start of each new day
# for x0 in mid_x_values:
#     fig.add_shape(type='line',
#                 x0=x0, 
#                 y0=0, 
#                 x1=x0, 
#                 y1=val_df['co2_emissions_intensity_for_consumed_electricity'].max(),
#                 line=dict(color='black', 
#                           width=3, 
#                           dash='dash')
#                 )

# # update layout
# fig.update_layout(title='When to Charge', 
#                   xaxis_title='UTC Time', 
#                   yaxis_title='Carbon Intensity',
#                   paper_bgcolor='rgba(0,0,0,0)',
#                   plot_bgcolor='rgba(0,0,0,0)',
#                   yaxis_visible=False, 
#                   yaxis_showticklabels=False,
#                   height=400,
#                   width=1400
#                   )

# fig.update_xaxes(tickvals=q1_quartile_df['utc_time'].tolist() + q4_quartile_df['utc_time'].tolist(),
#                  tickformat='%I%p', # set tick format
#                  tickangle=300
#                  )

# # Add text for weekday name and date (month and day)
# for i in range(len(mid_x_values)):
#     fig.add_annotation(x=mid_x_values[i], 
#                        y=.5,
#                        text=str(unique_weekdays[i])+'<br>'+str(unique_dates[i]),
#                        showarrow=False,
#                        xshift=40,
#                        yshift=10
#                       )
    
# # Create dummy markers to label legend    
# fig.add_scatter(x=[None],
#                 y=[None],
#                 mode="markers",
#                 name="Charge",
#                 marker=dict(size=7, 
#                             color="#6eb12e", 
#                             opacity=0.4, 
#                             symbol='circle')
#                )

# fig.add_scatter(x=[None],
#                 y=[None],
#                 mode="markers",
#                 name="Do NOT Charge",
#                 marker=dict(size=7, 
#                             color="red", 
#                             opacity=0.4, 
#                             symbol='circle')
#                )


# # show plot
# fig.show()


In [76]:
# %%time
# # This is the main graph window

# # Save actual, forecast, and forecast horizon vertical line traces separately
# actual_data, forecast_data, horizon_data = [], [], []
# initial_position = 182 # this is July 1st

# # Step through each daily dataframe (1 window corresponding to 1 slider step)
# # Add individual trades to the slider step graph for each day
# for d in range(0,len(list_of_dfs)):
#     carbon_day_df = list_of_dfs[d]
#     x = carbon_day_df.index
#     y = carbon_day_df['co2_emissions_intensity_for_consumed_electricity']
#     y2 = carbon_day_df['co2_forecast']
#     xf = carbon_day_df.index[-96]

#     actual_trace = go.Scatter(
#         x=x,
#         y=y,
#         name="Actual",
#         line=dict(color="black", width=2),
#         visible=False, )

#     forecast_trace = go.Scatter(
#         x=x,
#         y=y2,
#         name="Forecast",
#         line=dict(color="blue", width=2),
#         visible=False, )
    
#     horizon_trace = go.Scatter(
#         x=[xf, xf],
#         y=[0.2, 0.8],
#         mode='lines',
#         name='Horizon',
#         line=dict(color='grey', width=3, dash='dash'),
#         visible=False, )

#     actual_data.append(actual_trace)
#     forecast_data.append(forecast_trace)
#     horizon_data.append(horizon_trace)

# # Show the traces from the pair with index[initial position] when screen first loads
# actual_data[initial_position].visible = True
# forecast_data[initial_position].visible = True
# horizon_data[initial_position].visible = True

# data = actual_data + forecast_data + horizon_data

# steps = []
# """
# actual_data, forecast_data hidden separately, then combined and add to visible key
# For example, when the slider is 2,
# actual_visible = [False, False, True, False,...]
# forecast_visible = [False, False, True, False,...]
# By combining the two afterward, 2 traces can be displayed at once
# """
# # Create the actual graph object for each day
# for s in range(len(actual_data)):
#     # initially hide all data once
#     actual_visible, forecast_visible, horizon_visible = [False] * len(actual_data), [False] * len(forecast_data), [False] * len(horizon_data)
#     # then make only the sth traces visible
#     actual_visible[s], forecast_visible[s], horizon_visible[s] = True, True, True
#     step = dict(method="update",
#                 args=[{"visible": actual_visible + forecast_visible + horizon_visible},
#                       {"title":  "Carbon Intensity Forecast as of<br>"+forecasts_df.index[s].strftime('%B %-d, %Y')
#                                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                                   +"would reduce your carbon emissions by<br><br><br>"
#                                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[s][2]:.1%}</span>",
#                                   },
#                      ],
#                 label=forecasts_df.index[s].strftime('%b<br> %Y '),
#                 )
#     steps.append(step)

# sliders = [dict(active=initial_position,
#                         currentvalue={"prefix": "Selected Date:  ", "suffix":" test", "font": {"color": "white", "size":1}},
#                         pad=dict(t=40, b=200), # added padding to top and bottom so that text apppears above and below chart
#                         # font=dict(size=30),
#                         # color="white",
#                         minorticklen=2,
#                         steps=steps)]

# layout = go.Layout(
#     title=forecasts_df.index[initial_position].strftime('%B %d, %Y'),
#     # xaxis=dict(title="x"),
#     yaxis=dict(title="CO2 Intensity (lbs CO2/kWh)"),
#     font=dict(size=16),
#     hovermode='x unified',
#     hoverlabel=dict(font_size=16),
#     sliders=sliders,
#     showlegend=True)

# fig1 = go.Figure(data=data, layout=layout)
# fig1.update_xaxes(dict(
#     # type="date",
#     # tick0="2022-01-15 00:00:00",
#     # dtick="M1",
#     dtick="D1",
#     # tickformat="%b %d\n%Y"
#     tickformat="%-m/%-d\n%Y")
# )
# fig1.update_layout(
#     sliders=sliders,
#     title={'text': "Carbon Intensity Forecast as of<br>"+forecasts_df.index[initial_position].strftime('%B %-d, %Y')
#                   +"<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>"
#                   +"Consuming electricity at the lows vs. highs in this forecast window<br>"
#                   +"would reduce your carbon emissions by<br><br><br>"
#                   +f"<span style='font-size: 80px; font-weight: bold;'>{quartile_intensities[initial_position][2]:.1%}</span>",
#            'x': 0.46
#           },
#     width=1000,
#     height=1000
#     # sliders=[dict(
#         # tickformat='%B %d, %Y'
#     # )]
# )

# # Either show the plot or save it as an offline plot
# fig1.show()
# # offline.plot(fig, include_plotlyjs="cdn", filename="test_carbon_intensity_slider_plot.html")

In [77]:
# # Save chart as html

# # Set file name for saved chart
# file_name = 'Data Visualizations/test_carbon_intensity_slider_plot.html'

# # Write chart to html
# fig1.write_html(file_name, auto_play=False)